In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
#sns.set(style="darkgrid")

from scipy import stats as st

import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/DATASETS/Тестовые/Тест_Старший_аналитик.xlsx', sheet_name=0)

In [ ]:
df.head()

,Дивизион,Канал,Вывеска,Проект,Бренд,Брендобъем,Год,Месяц,План руб,Факт руб,План тонны,Факт тонны
0,ВОСТОК_ПРГ,ГОСЗАКУПКИ,NaN,ДП,МАЛЫШАМ,Сок МАЛЫШАМ 0.200 Тетра Пак,2013,Январь,6521069.72,8182954.10,261.674580,256.003254
1,ВОСТОК_ПРГ,ГОСЗАКУПКИ,NaN,ДП,МАЛЫШАМ,Сок МАЛЫШАМ 0.200 Тетра Пак,2013,Февраль,4795901.79,5632817.79,192.826769,183.638664
2,ВОСТОК_ПРГ,ГОСЗАКУПКИ,NaN,ДП,МАЛЫШАМ,Сок МАЛЫШАМ 0.200 Тетра Пак,2013,Март,10627862.78,3076383.72,390.499715,104.779083
3,ВОСТОК_ПРГ,ГОСЗАКУПКИ,NaN,ДП,МАЛЫШАМ,Сок МАЛЫШАМ 0.200 Тетра Пак,2013,Апрель,5891326.35,1886433.67,216.566416,66.719268
4,ВОСТОК_ПРГ,ГОСЗАКУПКИ,NaN,ДП,МАЛЫШАМ,Сок МАЛЫШАМ 0.200 Тетра Пак,2013,Май,4971125.78,1944910.39,182.794337,67.253220


In [ ]:
df.fillna(0.0, inplace=True)
df['План руб'] = df['План руб'] / 1000000
df['Факт руб'] = df['Факт руб'] / 1000000

def asd(month):
  if month == 'Январь':
    return 1
  if month == 'Февраль':
    return 2
  if month == 'Март':
    return 3
  if month == 'Апрель':
    return 4
  if month == 'Май':
    return 5

df['month'] = df['Месяц'].apply(asd)

# ЧАСТЬ 1

Вопрос №1

In [ ]:
# группируем данные
t = df[df['Год'] == 2014].groupby(['Год', 'month','Бренд']).agg({'Факт руб' : 'sum'}).reset_index()

t['Факт руб'] = t['Факт руб'].round(0)



# строим график
fig = px.bar(t.sort_values(by=['Год', 'month', 'Факт руб']), x='month', y='Факт руб', color='Бренд',\
            title='Фактические продажи брендов по месяцам, млн руб',\
             text ='Факт руб',
             labels={'month' : 'Номер Месяца'}
            )
fig.update_layout(barmode='group', xaxis_tickangle=0, title_font_family="Times New Roman")
fig.update_traces( textposition='outside', textangle=0)
fig.show()

Краткий вывод
1. Идет рост продаж от месяца к месяцу
2. Яркий лидер бренд - "Малышам"

In [ ]:
tt = df[df['Год'] == 2014].groupby(['Год', 'month','Бренд']).agg({'Факт руб' : 'sum', 'План руб' : 'sum'}).reset_index()
tt['Выполнение_плана'] = tt['Факт руб'] / tt['План руб'] -1
tt['Выполнение_плана'] = tt['Выполнение_плана'].round(2)
#tt

# график
fig = px.bar(tt, x='month', y='Выполнение_плана', color='Бренд',\
            title='Выполнения плана по брендам и по месяцам',\
             text ='Выполнение_плана',
             labels={'month' : 'Номер Месяца'}
            )
fig.update_layout(barmode='group', xaxis_tickangle=0, title_font_family="Times New Roman")
fig.update_traces( textposition='outside', textangle=0)
fig.show()

Краткий вывод
1. Почти по всем брендам и месяцам идет невыполнение плана, в среднем -15%
2. Достаточно сильное перевыполнение плана только в апреле у ЛБЮВЕТ0305 и в мае у ЛБЮВЕТ5.0

In [ ]:
t2 = df[df['Год'] == 2013].groupby(['Год', 'month','Бренд']).agg({'Факт руб' : 'sum'}).reset_index()

report = t.merge(t2, on=['month', 'Бренд'])
report['Прирост'] = (report['Факт руб_x'] / report['Факт руб_y'] -1 )
report.Прирост = report.Прирост.round(2)

In [ ]:
fig = px.bar(report, x='month', y='Прирост', color='Бренд',\
            title='Прирост продаж к прошлому году по брендам по месяцам',\
             text ='Прирост',
             labels={'month' : 'Номер Месяца'}
            )
fig.update_layout(barmode='group', xaxis_tickangle=0, title_font_family="Times New Roman")
fig.update_traces( textposition='outside', textangle=0)
fig.show()

Краткий вывод
1. В начале года отмечаем сильный прирост к прошлому году
2. В мае продажи +- равны маю 2013

Вопрос №2

In [ ]:
s = df.loc[df['Год'] == 2014,['Бренд', 'month','Факт руб']]
#s
df.loc[(df['Год'] == 2014) & (df['Бренд'] == 'МАЛЫШАМ'),['Факт руб']].sum()
df.query('Год==2014 and Бренд=="ЛБЮВЕТ0305"')['Факт руб'].sum()
ss = s.groupby(['month', 'Бренд']).sum().groupby('Бренд').cumsum().reset_index()

ss['Факт руб_'] = ss['Факт руб'].round(2)

In [ ]:
fig = px.bar(ss, x='month', y='Факт руб', color='Бренд',\
            title='Накопительные фактические продажи по брендов и по месяцам, млн руб',\
             text ='Факт руб_',
             labels={'month' : 'Номер Месяца'}
            )
fig.update_layout(barmode='stack', xaxis_tickangle=0, title_font_family="Times New Roman")
fig.update_traces( textposition='outside', textangle=0)
fig.show()

Краткий вывод
1. Доля брендов в динамике достаточно стабильна. Явный лидер - "Малыши"

In [ ]:
d = df.loc[df['Год'] == 2014,['Бренд', 'month','Факт руб', 'План руб']]
dd = d.groupby(['month', 'Бренд']).sum().groupby('Бренд').cumsum().reset_index()
#dd
dd['Выполнение плана'] = dd['Факт руб'] / dd['План руб'] - 1
dd['Выполнение плана'] = dd['Выполнение плана'].round(2)

In [ ]:
fig = px.bar(dd, x='month', y='Выполнение плана', color='Бренд',\
            title='Накопительное выполнение плана по брендам и по месяцам',\
             text ='Выполнение плана',
             labels={'month' : 'Номер Месяца'}
            )
fig.update_layout(barmode='group', xaxis_tickangle=0, title_font_family="Times New Roman")
fig.update_traces( textposition='outside', textangle=0)
fig.show()

Краткий вывод
1. Сильное невыполнение у брендов "Малыши" и "ЛБЮВЕТ5.0": >10%
2. Бренд ЛЮБЮВЕТ1.5 выполнил план

In [ ]:
s1 = df.loc[df['Год'] == 2013,['Бренд', 'month','Факт руб']]
ss1 = s1.groupby(['month', 'Бренд']).sum().groupby('Бренд').cumsum().reset_index()
rep = ss.merge(ss1, on=['month', 'Бренд'])
rep['Прирост'] = rep['Факт руб_x'] / rep['Факт руб_y'] - 1
rep['Прирост'] = rep['Прирост'].round(2)
#rep

In [ ]:
fig = px.bar(rep, x='month', y='Прирост', color='Бренд',\
            title='Прирост накопительных продаж по брендам к прошлому году',\
             text ='Прирост',
             labels={'month' : 'Номер Месяца'}
            )
fig.update_layout(barmode='group', xaxis_tickangle=0, title_font_family="Times New Roman")
fig.update_traces( textposition='outside', textangle=0)
fig.show()

Краткий вывод
1. По итогу мая накопительные продажи всех брендов, кроме ЛБЮВЕТ1.5, вышли в плюс по сравнению с прошлым годом

# ЧАСТЬ 2

Вопрос №1

In [ ]:
z = df[df['Год'] == 2014].groupby(['Год', 'month','Канал']).agg({'Факт руб' : 'sum'}).reset_index()

z['Факт руб'] = z['Факт руб'].round(2)


# график
fig = px.bar(z.sort_values(by=['month', 'Факт руб']), x='month', y='Факт руб', color='Канал',\
            title='Фактические продажи по каналам и по месяцам, млн руб',\
             text ='Факт руб',
             labels={'month' : 'Номер Месяца'}
            )
fig.update_layout(barmode='group', xaxis_tickangle=0, title_font_family="Times New Roman")
fig.update_traces( textposition='outside', textangle=0)
fig.show()

Краткий вывод
1. В январе было 2 явных лидера среди каналов сбыта: ФК и Дистрибьютор
2. в остальных месяцах лидер уже Дистрибьютор

In [ ]:
zz = df[df['Год'] == 2014].groupby(['Год','month','Канал']).agg({'Факт руб' : 'sum', 'План руб' : 'sum'}).reset_index()
zz['Выполнение_плана'] = zz['Факт руб'] / zz['План руб'] -1
zz['Выполнение_плана'] = zz['Выполнение_плана'].round(2)
#zz

# график
fig = px.bar(zz, x='month', y='Выполнение_плана', color='Канал',\
            title='Выполнения плана по каналам в динамике по месяцам, млн руб',\
             text ='Выполнение_плана',
             labels={'month' : 'Номер Месяца'}
            )
fig.update_layout(barmode='group', xaxis_tickangle=0, title_font_family="Times New Roman")
fig.update_traces( textposition='outside', textangle=0)
fig.show()

Краткой вывод
1. Фактически выполнение идет по каналам Физлицо, ФК и ЛК 

In [ ]:
z2 = df[df['Год'] == 2013].groupby(['Год', 'month','Канал']).agg({'Факт руб' : 'sum'}).reset_index()


r1 = z.merge(z2, on=['month', 'Канал'], how='outer')
#z.head()
#z2.head()
r1['Прирост'] = r1['Факт руб_x'] / r1['Факт руб_y'] -1
r1['Прирост'] = r1['Прирост'].round(2)
#r1

In [ ]:
fig = px.bar(r1.sort_values(by=['month', 'Прирост']), x='month', y='Прирост', color='Канал',\
            title='Прирост продаж к прошлому году по каналам в динамике по месяцам',\
             text ='Прирост',
             labels={'month' : 'Номер Месяца'}
            )
fig.update_layout(barmode='group', xaxis_tickangle=0, title_font_family="Times New Roman")
fig.update_traces( textposition='outside', textangle=0)
fig.show()

Краткий вывод
1. Сильный прирост идет по каналу Физлицо, ЛК и Horeca в мае 2014

Вопрос №2

In [ ]:
x = df.loc[df['Год'] == 2014, ['month','Канал', 'Факт руб']]
xx = x.groupby(['month', 'Канал']).sum().groupby('Канал').cumsum().reset_index()
xx['Факт руб_'] = xx['Факт руб'].round(2)

In [ ]:
fig = px.bar(xx.sort_values(by=['month','Факт руб']), x='month', y='Факт руб', color='Канал',\
            title='Накопительные фактические продажи по каналам и по месяцам, млн руб',\
             text ='Факт руб_',
             labels={'month' : 'Номер Месяца'}
            )
fig.update_layout(barmode='stack', xaxis_tickangle=0, title_font_family="Times New Roman")
fig.update_traces( textposition='outside', textangle=0)
fig.show()

Краткий вывод
1. Лидеры по каналам - Дистрибьютор, ФК и ЛК 

In [ ]:
c = df.loc[df['Год'] == 2014,['Канал', 'month','Факт руб', 'План руб']]
cc = c.groupby(['month', 'Канал']).sum().groupby('Канал').cumsum().reset_index()
cc['Выполнение_плана'] = cc['Факт руб'] / cc['План руб'] - 1
cc['Выполнение_плана'] = cc['Выполнение_плана'].round(2)

In [ ]:
fig = px.bar(cc.sort_values(by=['month', 'Факт руб']), x='month', y='Выполнение_плана', color='Канал',\
            title='Накопительное выполнение плана по каналам и по месяцам',\
             text ='Выполнение_плана',
             labels={'month' : 'Номер Месяца'}
            )
fig.update_layout(barmode='group', xaxis_tickangle=0, title_font_family="Times New Roman")
fig.update_traces( textposition='outside', textangle=0)
fig.show()

Краткий вывод
1. Выполнение у каналов Horeca, Госзакупки и ФК
2. Сильный провал в канале Розница, -34%

In [ ]:
v = df.loc[df['Год'] == 2013,['Канал', 'month','Факт руб']]
vv = v.groupby(['month', 'Канал']).sum().groupby('Канал').cumsum().reset_index()
re = xx.merge(vv, on=['month', 'Канал'], how='outer')
re['Прирост'] = re['Факт руб_x'] / re['Факт руб_y'] - 1
re['Прирост'] = re['Прирост'].round(2)

In [ ]:
fig = px.bar(re, x='month', y='Прирост', color='Канал',\
            title='Прирост накопительных продаж к прошлому году по каналам',\
             text ='Прирост',
             labels={'month' : 'Номер Месяца'}
            )
fig.update_layout(barmode='group', xaxis_tickangle=0, title_font_family="Times New Roman")
fig.update_traces( textposition='outside', textangle=0)
fig.show()

Краткий вывод
1. Сильный прирост по каналам Физлицо, ЛК и ФК

Общий вывод

Бренды:
1. Самые продаваемые бренды: "Малыши" и "ЛБЮВЕТ5.0" и показывают положительную динамику с 2013 годом
2. Невыполнение плана по лидерам брендов около 10-12%

Каналы:
1. Самые продаваемые каналы: Дистрибьютор и ФК
2. Сильное накопительное невыполнение по каналам Розница (-34%) и Дистрибьютор (-15%)
3. Сильный прирост по сравнению с 2013 годом в каналах Физлицо (+220%), ЛК (+50%), Прочее (+39%), Horeca (+24%)
4. Сильное падение к 2013 году по каналу Розница (-96%) и Госзакупки (-74%)
